In [ ]:
import keras
from keras.models import load_model
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.layers import Dense, Activation
from numpy import argmax
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing import image
import tensorflow as tf
from keras.applications.resnet50 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from skimage.io import imread
import cv2
import matplotlib.pyplot as plt
import tkinter
import _tkinter
import threading
from threading import Thread
import warnings
warnings.filterwarnings('ignore')
tf.compat.v1.disable_eager_execution()
import traceback
import contextlib
from tkinter import *
import sys
from PIL import Image, ImageChops, ImageTk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk, filedialog, messagebox
from sklearn.preprocessing import Binarizer
import tkinter as tk
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import time
import sys
import tkinter.font

#다양한 기능

# 폴더 내 파일 삭제 기능
import os 
def removeAllFile(filePath):
    if os.path.exists(filePath):
        for file in os.scandir(filePath):
            os.remove(file.path)
        return 
    else:
        return 


#파일 삭제 전, 경고문 보여주기
def warning():
    MsgBox =tk.messagebox.askquestion("Warning", "폴더 내 모든 파일을 삭제하시겠습니까?",icon='warning')
    if MsgBox == 'yes':
        removeAllFile('./Image_File/')
        

    
# 검사할 마스크, 설비 옵션 선택
def inspection():
    font=tkinter.font.Font(family="Courier", size=20, weight="bold")
    label = Label(window, text="마스크 종류:",width=10,height=3,bg='gray40',font=font)   
    label.place(x=50, y=300)  
    
    
    fontE = ("Courier", 20, "bold")
    mb = StringVar()                                         
    mbn = ttk.Combobox(window, width=15, textvariable=mb,font=fontE  ) 
    mbn['values'] = ("KF-94", "New KF-94","KF-80", "KF-AD")                      
    mbn.place(x=250, y=335)

    
    label1 = Label(window, text="가동 설비:",width=10,height=3,bg='gray40',font=font)   
    label1.place(x=50, y=500)  


    mb1 = StringVar()                                         
    mbn1 = ttk.Combobox(window, width=15, textvariable=mb1,font=fontE)   
    mbn1['values'] = ("1호기","2호기","3호기","4호기","5호기","6호기","7호기","8호기","9호기","10호기")                      
    mbn1.place(x=250, y=535)

    
    button=Button(window,fg="blue",bg="gray",font=fontE  ,padx=50,pady=50,text="START", command=callback)
    button.place(x=700,y=380)
    
    
    font=tkinter.font.Font(family="Courier", size=20, weight="bold")
    butt4 = Button(window,text="판정 결과",width=45,height=2,font=font,fg="white",bg="gray")
    butt4.place(x=1100, y=200)




def callback():

    
    class Target:
        watchDir = "./Image_File"
    #watchDir에 감시하려는 디렉토리를 명시

        def __init__(self):
            self.observer = Observer()   #observer객체를 만듦

        def run(self):
            event_handler = Handler()
            self.observer.schedule(event_handler, self.watchDir, recursive=True)
            self.observer.start()
            try:
                while True:
                    time.sleep(1)
            except:
                self.observer.stop()
                self.observer.join()
                
    class Handler(FileSystemEventHandler):
        def on_created(self, event):  # 파일, 디렉터리가 생성되면 실행
            filepath = event.src_path
            filepath = os.path.dirname(filepath)
            
            filename = event.src_path
            filename = os.path.basename(filename)
            
            a = os.path.join(filepath, filename)
            
            # 이미지데이터 불러오기
            image = cv2.imread(a, 0)
            
            # 불러온 이미지 데이터를 학습하였을 당시, 전처리 과정과 동일한 조건으로 처리 
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            cll = clahe.apply(image)
            kernel = np.array([[-0.8, 0, 0.8],
                               [0, 1, 0],
                               [-0.8, 0, 0.8]])  # 커널을 만듬
            img = cv2.filter2D(cll, -1, kernel)
            img = img[5:1750, 1545:2280]
            img = cv2.resize(img, dsize=(0, 0), fx=0.086, fy=0.086, interpolation=cv2.INTER_AREA) # 이미지 resize
            image = cv2.rectangle(img, (0, 22), (110, 125), (0, 0, 0), -1)
            img= Image.fromarray(image)
            img_size = img.size  # 이미지의 크기 측정
            x = img_size[0]  # 넓이값
            y = img_size[1]  # 높이값
            if x != y:
                size = max(x, y)
                resized_img = Image.new(mode='RGB', size=(size, size), color=('black'))
                offset = (round((abs(x - size)) / 2), round((abs(y - size)) / 2))
                resized_img.paste(img, offset)
                image = img_to_array(resized_img)
                image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
 

                pre_datagen = ImageDataGenerator(rescale=1. / 255)
                pre_generator = pre_datagen.flow(
                    image,
                    y=None,
                    batch_size=1,
                )
                model = load_model("best3.h5") # 분류모델 load
                predictions = model.predict_generator(pre_generator, verbose=1)
                predictions_thres = predictions[:, 1].reshape(-1, 1)
                binarizer = Binarizer(threshold= 0.55).fit(predictions_thres)
                custom_predict = binarizer.transform(predictions_thres)
                custom_predict = custom_predict.tolist()
                a = [[0.0]] 
                if custom_predict == a: # 판정 결과 양품 
                    
                    root = Tk()

                    root.geometry("500x400-250+400")
                    root.attributes("-topmost", 1)
                    lbl = Label(root, text='Ok',bg = "gray40", width = 3, height = 1,font = ("궁서체",300), fg ="blue")
                    lbl.place(x=0,y=0)
                    root.overrideredirect(True)
                    root.after(700, root.destroy)
                    root.mainloop()

                else: # 판정 결과 불량품 
                    root1 = Tk()
                    root1.geometry("500x400-250+400")
                    root1.attributes("-topmost", 1)
                    lbl = Label(root1, text='No',bg = "gray40", width = 3, height = 1,font = ("궁서체",300), fg ="red")
                    lbl.place(x=0,y=0)
                    root1.overrideredirect(True)
                    root1.after(700, root1.destroy)
                    root1.mainloop()

                        
                        
    if __name__ == "__main__" :#본 파일에서 실행될 때만 실행되도록 함
        print('Sites folder watchdog is running...')
        w = Target()
        w.run()
# exe 파일로 실행하기 위함 (GUI Setting) 
window =Tk()
window.title('MasK Inspection Program')

window.geometry('2000x800+0+0')


butt = Button(window,text="Mask Inspection Program",width=83,height=1,font=("Courier", 30, "bold"),fg="black",bg="yellow")
butt.place(x=0, y=0)

butt0 = Canvas(window,width=10000,height=10000,bg="gray40")
butt0.place(x=0, y=88)
butt0.create_line(1000,0,1000,100000, fill='white')

butt1 = Canvas(window,width=10000,height=50,bg="gray40")
butt1.place(x=0, y=88)
butt1.create_line(1000,0,1000,100, fill='white')
fontt=tkinter.font.Font(family="Courier", size=9, weight="bold")
butt2 = Button(window,text="Model : VGG16",width=20,height=2,fg="white",bg="gray",font=fontt)
butt2.place(x=1, y=95)

font=tkinter.font.Font(family="Courier", size=20, weight="bold")
butt3 = Button(window,text="마스크 검사 설정",width=50,height=2,font=font,fg="white",bg="gray",command=inspection)
butt3.place(x=100, y=200)

fonttt=tkinter.font.Font(family="Courier", size=18, weight="bold")
button = Button(window,text="Reset File",width=11,height=0,fg="white",bg="red",font=fonttt,command=warning)
button.place(x=1745, y=93)


window.mainloop()

# 다양한 기능

In [ ]:
# 폴더 내 파일 삭제 
import os 
def removeAllFile(filePath):
    if os.path.exists(filePath):
        for file in os.scandir(filePath):
            os.remove(file.path)
        return 
    else:
        return 


#파일 삭제 전, 경고문
def warning():
    MsgBox =tk.messagebox.askquestion("Warning", "폴더 내 모든 파일을 삭제하시겠습니까?",icon='warning')
    if MsgBox == 'yes':
        removeAllFile('./Image_File/')
        

    
    
def inspection():
    font=tkinter.font.Font(family="Courier", size=20, weight="bold")
    label = Label(window, text="마스크 종류:",width=10,height=3,bg='gray40',font=font)   
    label.place(x=50, y=300)  
    
    
    fontE = ("Courier", 20, "bold")
    mb = StringVar()                                         # String variable
    mbn = ttk.Combobox(window, width=15, textvariable=mb,font=fontE  ) # Create a combobox
    mbn['values'] = ("KF-94", "New KF-94","KF-80", "KF-AD")                      # Combobox's items
    mbn.place(x=250, y=335)

    
    label1 = Label(window, text="가동 설비:",width=10,height=3,bg='gray40',font=font)   
    label1.place(x=50, y=500)  


    mb1 = StringVar()                                         # String variable
    mbn1 = ttk.Combobox(window, width=15, textvariable=mb1,font=fontE)   # Create a combobox
    mbn1['values'] = ("1호기","2호기","3호기","4호기","5호기","6호기","7호기","8호기","9호기","10호기")                      # Combobox's items
    mbn1.place(x=250, y=535)

    
    button=Button(window,fg="blue",bg="gray",font=fontE  ,padx=50,pady=50,text="START", command=callback)
    button.place(x=700,y=380)
    
#     butt2 = Canvas(window,width=418,height=100,bg="gray40")
#     butt2.place(x=200, y=670)
#     butt2.create_line(1000,0,1000,100000, fill='white')

#     butt3 = Canvas(window,width=418,height=100,bg="gray40")
#     butt3.place(x=200, y=870)


#     butt3.create_line(205,0,205,110, fill='white')
    
#     font=tkinter.font.Font(family="Courier", size=20, weight="bold")
#     la = Label(window, text="판정 결과",width=9,height=2,bg='gray',fg='white',font=font)   
#     la.place(x=1400, y=200)  
    
    font=tkinter.font.Font(family="Courier", size=20, weight="bold")
    butt4 = Button(window,text="판정 결과",width=45,height=2,font=font,fg="white",bg="gray")
    butt4.place(x=1100, y=200)

#     font=tkinter.font.Font(family="Courier", size=12, weight="bold")
#     la1 = Label(window, text="검사 수량",width=10,height=3,bg='white',font=font)   
#     la1.place(x=350, y=600)

#     font=tkinter.font.Font(family="Courier", size=12, weight="bold")
#     la2 = Label(window, text=" 양품 | 불량 ",width=15,height=3,bg='white',font=font)   
#     la2.place(x=325, y=800)

#     font=tkinter.font.Font(family="Courier", size=30, weight="bold")
#     la3 = tkinter.Label(window, text="0",width=3,height=1,fg='black',bg='gray40',font=font)   
#     la3.place(x=360, y=700)
        
#     font=tkinter.font.Font(family="Courier", size=30, weight="bold")
#     la3 = tkinter.Label(window, text="0",width=3,height=1,fg='blue',bg='gray40',font=font)   
#     la3.place(x=270, y=900)
#     font=tkinter.font.Font(family="Courier", size=30, weight="bold")
#     la3 = tkinter.Label(window, text="0",width=3,height=1,fg='red',bg='gray40',font=font)   
#     la3.place(x=470, y=900)

In [ ]:
# 실제 사용할 것, 2021-05-23

def callback():
#     button["text"] = '검사를 마치겠습니다.'
    
    class Target:
        watchDir = "./Image_File"
    #watchDir에 감시하려는 디렉토리를 명시한다.

        def __init__(self):
            self.observer = Observer()   #observer객체를 만듦

        def run(self):
            event_handler = Handler()
            self.observer.schedule(event_handler, self.watchDir, recursive=True)
            self.observer.start()
            try:
                while True:
                    time.sleep(1)
            except:
                self.observer.stop()
                self.observer.join()
                
    class Handler(FileSystemEventHandler):
        def on_created(self, event):  # 파일, 디렉터리가 생성되면 실행
            filepath = event.src_path
            filepath = os.path.dirname(filepath)
            
            filename = event.src_path
            filename = os.path.basename(filename)
            
            a = os.path.join(filepath, filename)
            
            image = cv2.imread(a, 0)
            
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            cll = clahe.apply(image)
            kernel = np.array([[-0.8, 0, 0.8],
                               [0, 1, 0],
                               [-0.8, 0, 0.8]])  # 커널을 만듭니다.
            img = cv2.filter2D(cll, -1, kernel)
            img = img[5:1750, 1545:2280]
            img = cv2.resize(img, dsize=(0, 0), fx=0.086, fy=0.086, interpolation=cv2.INTER_AREA)
            image = cv2.rectangle(img, (0, 22), (110, 125), (0, 0, 0), -1)
            img= Image.fromarray(image)
            img_size = img.size  # 이미지의 크기 측정
            x = img_size[0]  # 넓이값
            y = img_size[1]  # 높이값
            if x != y:
                size = max(x, y)
                resized_img = Image.new(mode='RGB', size=(size, size), color=('black'))
                offset = (round((abs(x - size)) / 2), round((abs(y - size)) / 2))
                resized_img.paste(img, offset)
                image = img_to_array(resized_img)
                image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    #             image = preprocess_input(image)

                pre_datagen = ImageDataGenerator(rescale=1. / 255)
                pre_generator = pre_datagen.flow(
                    image,
                    y=None,
                    batch_size=1,
                )
                model = load_model("best3.h5")
                predictions = model.predict_generator(pre_generator, verbose=1)
                predictions_thres = predictions[:, 1].reshape(-1, 1)
                binarizer = Binarizer(threshold= 0.55).fit(predictions_thres)
                custom_predict = binarizer.transform(predictions_thres)
                custom_predict = custom_predict.tolist()
                a = [[0.0]]
                if custom_predict == a:    
                    
                    root = Tk()

                    root.geometry("500x400-250+400")
                    root.attributes("-topmost", 1)
                    lbl = Label(root, text='Ok',bg = "gray40", width = 3, height = 1,font = ("궁서체",300), fg ="blue")
                    lbl.place(x=0,y=0)
                    root.overrideredirect(True)
                    root.after(700, root.destroy)
                    root.mainloop()

                else:
                    root1 = Tk()
                    root1.geometry("500x400-250+400")
                    root1.attributes("-topmost", 1)
                    lbl = Label(root1, text='No',bg = "gray40", width = 3, height = 1,font = ("궁서체",300), fg ="red")
                    lbl.place(x=0,y=0)
                    root1.overrideredirect(True)
                    root1.after(700, root1.destroy)
                    root1.mainloop()

                        
                        
    if __name__ == "__main__" :#본 파일에서 실행될 때만 실행되도록 함
        print('Sites folder watchdog is running...')
        w = Target()
        w.run()


# main

In [ ]:
window =Tk()
window.title('MasK Inspection Program')

window.geometry('2000x800+0+0')


butt = Button(window,text="Mask Inspection Program",width=83,height=1,font=("Courier", 30, "bold"),fg="black",bg="yellow")
butt.place(x=0, y=0)

butt0 = Canvas(window,width=10000,height=10000,bg="gray40")
butt0.place(x=0, y=88)
butt0.create_line(1000,0,1000,100000, fill='white')

butt1 = Canvas(window,width=10000,height=50,bg="gray40")
butt1.place(x=0, y=88)
butt1.create_line(1000,0,1000,100, fill='white')
fontt=tkinter.font.Font(family="Courier", size=9, weight="bold")
butt2 = Button(window,text="Model : VGG16",width=20,height=2,fg="white",bg="gray",font=fontt)
butt2.place(x=1, y=95)

font=tkinter.font.Font(family="Courier", size=20, weight="bold")
butt3 = Button(window,text="마스크 검사 설정",width=50,height=2,font=font,fg="white",bg="gray",command=inspection)
butt3.place(x=100, y=200)

fonttt=tkinter.font.Font(family="Courier", size=18, weight="bold")
button = Button(window,text="Reset File",width=11,height=0,fg="white",bg="red",font=fonttt,command=warning)
button.place(x=1745, y=93)


window.mainloop()